In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [14]:
data_folder = 'raw_data'
raw_data_paths = [
    ('/depression', '/targets/DPQ_L_Target_Depression.xpt'),
    ('/insomnia', '/targets/SLQ_L_Target_Insomnia.xpt'),
]

for i in range(len(raw_data_paths)):

    data = pd.read_sas(data_folder + raw_data_paths[i][1], format='xport')

    # Remove rows with any missing values in the targets
    data = data[data[list(data.columns)].notnull().all(1)]

    for file_path in os.listdir(data_folder + raw_data_paths[i][0]):
        if not file_path.endswith('.xpt'):
            raise ValueError("Unsupported file format")

        file_path = os.path.join(data_folder + raw_data_paths[i][0], file_path)
        new_data = pd.read_sas(file_path, format='xport') 

        if 'SEQN' not in new_data.columns:
            print("Missing SEQN column in file: " + file_path)
            #print(f'ID: {new_data["SP_ID"].head()}')
        else:     
            data = pd.merge(data, new_data, how="left", left_on="SEQN", right_on="SEQN")

    numeric_cols = data.select_dtypes(include=['float64']).columns
    for col in numeric_cols:
        data[col] = data[col].round().astype("Int64")

    #print(f'SEQN: {data["SEQN"].head()}')
    os.makedirs('processed_data', exist_ok=True)
    data.to_csv('processed_data/' + raw_data_paths[i][0] + '_data.csv', index=False)

Missing SEQN column in file: raw_data/depression\VTQ_L.xpt
Missing SEQN column in file: raw_data/insomnia\VTQ_L.xpt


In [15]:
file_path = os.path.join(data_folder + raw_data_paths[0][0], 'VTQ_L.xpt')
new_data = pd.read_sas(file_path, format='xport') 
print(new_data.columns)
print(new_data['SP_ID'])

Index(['SP_ID', 'WTSVOC2Y', 'VTQ231A', 'VTQ233A', 'VTQ233B', 'VTQ244A',
       'VTD244B', 'VTQ261A', 'VTD261B', 'VTQ271A', 'VTD271B', 'VTQ281A',
       'VTD281B'],
      dtype='object')
0       100151.0
1       100331.0
2       100457.0
3       100490.0
4       100727.0
5       101081.0
6       101153.0
7       101450.0
8       101746.0
9       102281.0
10      103240.0
11      103769.0
12      103793.0
13      103978.0
14      104033.0
15      104747.0
16      105088.0
17      105226.0
18      105505.0
19      105580.0
20      105628.0
21      106054.0
22      106091.0
23      106133.0
24      106693.0
25      106706.0
26      106762.0
27      106853.0
28      107473.0
29      107749.0
30      108025.0
31      108310.0
32      108353.0
33      108719.0
34      108760.0
35      108796.0
36      108950.0
37      109102.0
38      109168.0
39      109240.0
40      109525.0
41      109574.0
42      110045.0
43      110068.0
44      110149.0
45      110998.0
46      111226.0
47      111470.

In [16]:
import pandas as pd
import os

# Display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# Load depression data
df_depression = pd.read_csv('processed_data/depression_data.csv')

# Create a summary of missing and non-missing values
missing_summary = pd.DataFrame({
    'Missing Values': df_depression.isnull().sum(),
    'Non-Missing Values': df_depression.notnull().sum(),
    'Missing %': (df_depression.isnull().sum() / len(df_depression) * 100).round(2)
})

# Ensure output folder exists and save summary to files
os.makedirs('processed_data', exist_ok=True)
missing_summary.to_csv('processed_data/depression_missing_summary.csv')

print("=" * 70)
print("DEPRESSION DATA - MISSING VALUES SUMMARY")
print("=" * 70)
print(f"\nTotal rows: {len(df_depression)}")
print(f"Total columns: {len(df_depression.columns)}\n")
print(missing_summary)
print("\n" + "=" * 70)
print(f"Total missing values in dataset: {df_depression.isnull().sum().sum()}")
print("=" * 70)
print("Saved missing-summary to processed_data/depression_missing_summary.csv and .txt")

DEPRESSION DATA - MISSING VALUES SUMMARY

Total rows: 4167
Total columns: 265

          Missing Values  Non-Missing Values  Missing %
SEQN                   0                4167       0.00
DPQ010                 0                4167       0.00
DPQ020                 0                4167       0.00
DPQ030                 0                4167       0.00
DPQ040                 0                4167       0.00
DPQ050                 0                4167       0.00
DPQ060                 0                4167       0.00
DPQ070                 0                4167       0.00
DPQ080                 0                4167       0.00
DPQ090                 0                4167       0.00
DPQ100                 0                4167       0.00
ACD010A              767                3400      18.41
ACD010B             4158                   9      99.78
ACD010C             4000                 167      95.99
ACD040              3496                 671      83.90
ALQ111                20 

In [ ]:
import pandas as pd
import os

# Load merged depression data
df = pd.read_csv('processed_data/depression_data.csv')

# Prepare output folder
os.makedirs('processed_data', exist_ok=True)

# Print unique values and counts for each column (excluding SEQN) and save a detailed CSV
records = []
print('='*80)
print('VALUE COUNTS & UNIQUE VALUES FOR EACH COLUMN (excluding SEQN)')
print('='*80)
cols = [c for c in df.columns if c != 'SEQN']
for col in cols:
    print(f"\nColumn: {col}")
    vc = df[col].value_counts(dropna=False, sort=False)
    # Print the counts to the notebook
    print(vc)
    # Also show the distinct values
    unique_vals = df[col].dropna().unique()
    print(f"Unique (non-null) values: {list(unique_vals)}")

    # Record rows for CSV
    for val, cnt in vc.items():
        records.append({
            'column': col,
            'value': val,
            'count': int(cnt),
            'percent': round(int(cnt) / len(df) * 100, 2)
        })

vc_df = pd.DataFrame(records)
vc_df.to_csv('processed_data/depression_value_counts.csv', index=False)

print('\nSaved detailed value counts to processed_data/depression_value_counts.csv and .txt (excluding SEQN)')

VALUE COUNTS & UNIQUE VALUES FOR EACH COLUMN (excluding SEQN)

Column: DPQ010
DPQ010
0    2363
3     253
1    1202
2     334
7       3
9      12
Name: count, dtype: int64
Unique (non-null) values: [np.int64(0), np.int64(3), np.int64(1), np.int64(2), np.int64(7), np.int64(9)]

Column: DPQ020
DPQ020
0    2330
3     212
1    1323
2     294
9       6
7       2
Name: count, dtype: int64
Unique (non-null) values: [np.int64(0), np.int64(3), np.int64(1), np.int64(2), np.int64(9), np.int64(7)]

Column: DPQ030
DPQ030
1    1686
0    1433
3     534
2     510
9       2
7       2
Name: count, dtype: int64
Unique (non-null) values: [np.int64(1), np.int64(0), np.int64(3), np.int64(2), np.int64(9), np.int64(7)]

Column: DPQ040
DPQ040
0    1009
1    2110
3     472
2     570
9       3
7       3
Name: count, dtype: int64
Unique (non-null) values: [np.int64(0), np.int64(1), np.int64(3), np.int64(2), np.int64(9), np.int64(7)]

Column: DPQ050
DPQ050
0    2368
3     289
1    1136
2     371
9       3
Name: cou